In [ ]:
print("Insert nautical python to port distance")

In [1]:
import pandas.io.sql as sqlio
import psycopg2 as ps
import os
from dotenv import load_dotenv

In [2]:
import pandas as pd
from geopy.distance import geodesic as gd
import numpy as np


In [3]:
import nautical_calculations
from nautical_calculations.basic import get_distance
from nautical_calculations.operations import convert_to_miles
import numpy as np
from geopy.distance import geodesic as gd
import math
import haversine as hs
from haversine import Unit

In [4]:
load_dotenv()
db = os.getenv('POSTGRES_DB')
user = os.getenv('POSTGRES_USER')
password = os.getenv('POSTGRES_PASSWORD')
host = os.getenv('POSTGRES_SERVER')
port = os.getenv('POSTGRES_PORT')

In [5]:
conn = ps.connect(dbname=db, 
                  user=user,
                  password=password, 
                  host=host,
                  port=port)

In [16]:
# Accommodate query
sql = """SELECT id_port, port_name, code, latitude, longitude
FROM stg_pelni.master_port
where latitude is not null or longitude is not null"""

In [17]:
# Read Query SQL
df = sqlio.read_sql_query(sql, conn)

C:\Users\BN001602001\AppData\Local\Temp\ipykernel_5564\2377376051.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = sqlio.read_sql_query(sql, conn)


In [18]:
# Change name column
df = df.rename(columns={'code': 'Origin'})
df


,id_port,port_name,Origin,latitude,longitude
0,c672024c-821d-4c32-b162-612017b50b4c,Bawean,590,None,None
1,00739854-654f-469c-b5af-31680f7ae36e,Labuan Bajo,662,-8.493639,119.87533
2,13ae6ff9-3a38-42f4-bf51-ac03c0492d30,Amahai,952,-3.338247,128.91997
3,0c87bc40-0218-4bc1-bbd8-3dfff430a3d9,Karatung,845,4.722646,127.08841
4,ddac0441-65ff-4960-b993-87c73ebc2d85,Belinyu,322,-1.62999,105.733
...,...,...,...,...,...
92,3bd1c07f-44b1-4fb8-becc-e6faa9882fb3,Merauke,979,-8.477812,140.3904
93,4746d329-6f16-4e45-9cbc-7245ef40d5d6,Selat Lampa,725,-5.65,132.99
94,72346087-6a6a-462b-a434-913d6f2e5ca9,Ternate,942,0.780123,127.38839
95,48e4149c-db11-4979-af3b-c45ee6b94a51,Pomako,983,-4.4161277,136.93094


In [19]:
# Read CSV
df2 = pd.read_csv("data/Data_od.csv")
df2.head()

,Origin,Destination
0,891,697
1,891,908
2,891,845
3,891,681
4,891,322


In [20]:
# Change Typedate
df['Origin'] = df['Origin'].astype(int)
df2['Origin'] = df2['Origin'].astype(int)

# Inner Join
inner_join = pd.merge(df, 
                      df2, 
                      on ='Origin', 
                      how ='inner')

In [21]:
# Checked data
inner_join.loc[inner_join['Origin']==920]

,id_port,port_name,Origin,latitude,longitude,Destination


In [22]:
# Change name Columns
inner_join = inner_join.rename(columns={'id_port': 'port_origin'})
inner_join = inner_join.rename(columns={'port_name': 'port_name_origin'})
inner_join = inner_join.rename(columns={'latitude': 'lat_origin'})
inner_join = inner_join.rename(columns={'longitude': 'long_origin'})

df = df.rename(columns={'Origin': 'Destination'})

In [23]:
# Change Typedata
df['Destination'] = df['Destination'].astype(int)
df2['Destination'] = df2['Destination'].astype(int)

# Inner Join
merge_data = pd.merge(df, 
                      inner_join, 
                      on ='Destination', 
                      how ='inner')

In [24]:
# Change name columns
merge_data = merge_data.rename(columns={'id_port': 'port_destination'})
merge_data = merge_data.rename(columns={'port_name': 'port_name_destination'})
merge_data = merge_data.rename(columns={'latitude': 'lat_dest'})
merge_data = merge_data.rename(columns={'longitude': 'long_dest'})
merge_data

,port_destination,port_name_destination,Destination,lat_dest,long_dest,port_origin,port_name_origin,Origin,lat_origin,long_origin
0,c672024c-821d-4c32-b162-612017b50b4c,Bawean,590,None,None,00739854-654f-469c-b5af-31680f7ae36e,Labuan Bajo,662,-8.493639,119.87533
1,c672024c-821d-4c32-b162-612017b50b4c,Bawean,590,None,None,13ae6ff9-3a38-42f4-bf51-ac03c0492d30,Amahai,952,-3.338247,128.91997
2,c672024c-821d-4c32-b162-612017b50b4c,Bawean,590,None,None,0c87bc40-0218-4bc1-bbd8-3dfff430a3d9,Karatung,845,4.722646,127.08841
3,c672024c-821d-4c32-b162-612017b50b4c,Bawean,590,None,None,ddac0441-65ff-4960-b993-87c73ebc2d85,Belinyu,322,-1.62999,105.733
4,c672024c-821d-4c32-b162-612017b50b4c,Bawean,590,None,None,9c1e061d-9b9f-404d-a4cc-c98db3337a64,Namlea,951,-3.269646,127.08712
...,...,...,...,...,...,...,...,...,...,...
9307,1334ec95-cc8a-4464-a263-0b0beee559da,Sei Kolak Kijang,251,0.860583,104.6115,c069f8c4-4a93-4021-9a75-c9aadb3b4a1c,Banda Naira,947,-4.524874,129.8967
9308,1334ec95-cc8a-4464-a263-0b0beee559da,Sei Kolak Kijang,251,0.860583,104.6115,3bd1c07f-44b1-4fb8-becc-e6faa9882fb3,Merauke,979,-8.477812,140.3904
9309,1334ec95-cc8a-4464-a263-0b0beee559da,Sei Kolak Kijang,251,0.860583,104.6115,4746d329-6f16-4e45-9cbc-7245ef40d5d6,Selat Lampa,725,-5.65,132.99
9310,1334ec95-cc8a-4464-a263-0b0beee559da,Sei Kolak Kijang,251,0.860583,104.6115,72346087-6a6a-462b-a434-913d6f2e5ca9,Ternate,942,0.780123,127.38839


In [26]:
# Cleansing from data status 'None'
cleandf = merge_data[~((merge_data['lat_origin']=='None') | (merge_data['long_origin']=='None') | (merge_data['lat_dest']=='None') | (merge_data['long_dest']=='None'))]
cleandf

,port_destination,port_name_destination,Destination,lat_dest,long_dest,port_origin,port_name_origin,Origin,lat_origin,long_origin
97,00739854-654f-469c-b5af-31680f7ae36e,Labuan Bajo,662,-8.493639,119.87533,13ae6ff9-3a38-42f4-bf51-ac03c0492d30,Amahai,952,-3.338247,128.91997
98,00739854-654f-469c-b5af-31680f7ae36e,Labuan Bajo,662,-8.493639,119.87533,0c87bc40-0218-4bc1-bbd8-3dfff430a3d9,Karatung,845,4.722646,127.08841
99,00739854-654f-469c-b5af-31680f7ae36e,Labuan Bajo,662,-8.493639,119.87533,ddac0441-65ff-4960-b993-87c73ebc2d85,Belinyu,322,-1.62999,105.733
100,00739854-654f-469c-b5af-31680f7ae36e,Labuan Bajo,662,-8.493639,119.87533,9c1e061d-9b9f-404d-a4cc-c98db3337a64,Namlea,951,-3.269646,127.08712
103,00739854-654f-469c-b5af-31680f7ae36e,Labuan Bajo,662,-8.493639,119.87533,b2811ee3-48f4-41a3-b82a-bcd88ef3233c,Ambon,946,-3.693357,128.1766
...,...,...,...,...,...,...,...,...,...,...
9307,1334ec95-cc8a-4464-a263-0b0beee559da,Sei Kolak Kijang,251,0.860583,104.6115,c069f8c4-4a93-4021-9a75-c9aadb3b4a1c,Banda Naira,947,-4.524874,129.8967
9308,1334ec95-cc8a-4464-a263-0b0beee559da,Sei Kolak Kijang,251,0.860583,104.6115,3bd1c07f-44b1-4fb8-becc-e6faa9882fb3,Merauke,979,-8.477812,140.3904
9309,1334ec95-cc8a-4464-a263-0b0beee559da,Sei Kolak Kijang,251,0.860583,104.6115,4746d329-6f16-4e45-9cbc-7245ef40d5d6,Selat Lampa,725,-5.65,132.99
9310,1334ec95-cc8a-4464-a263-0b0beee559da,Sei Kolak Kijang,251,0.860583,104.6115,72346087-6a6a-462b-a434-913d6f2e5ca9,Ternate,942,0.780123,127.38839


In [27]:
cleandf = cleandf.dropna(subset=['lat_origin'])
cleandf = cleandf.dropna(subset=['long_origin'])

cleandf = cleandf.dropna(subset=['lat_dest'])
cleandf = cleandf.dropna(subset=['long_dest'])

In [29]:
# Change Typedata
cleandf['lat_origin'] = cleandf['lat_origin'].astype(float)
cleandf['long_origin'] = cleandf['long_origin'].astype(float)

cleandf['lat_dest'] = cleandf['lat_dest'].astype(float)
cleandf['long_dest'] = cleandf['long_dest'].astype(float)

cleandf.dtypes

port_destination          object
port_name_destination     object
Destination                int32
lat_dest                 float64
long_dest                float64
port_origin               object
port_name_origin          object
Origin                     int32
lat_origin               float64
long_origin              float64
dtype: object

In [30]:
# Calculate from latitude and longtitude data convert to miles (use other library)
nautical_data2 = []
for index, row in cleandf.iterrows():
    ori = (row['lat_origin'],row['long_origin'])
    des = (row['lat_dest'],row['long_dest'])
    rs = gd(ori, des).miles
    rs = round(rs)
    nautical_data2.append(rs)
    
# Add new columns    
cleandf['nautical'] = nautical_data2
cleandf['commercial'] = nautical_data2

In [31]:
# Change name columns
cleandf = cleandf.rename(columns={'Destination': 'code_destination'})
cleandf = cleandf.rename(columns={'Origin': 'code_origin'})
cleandf.columns.values

array(['port_destination', 'port_name_destination', 'code_destination',
       'lat_dest', 'long_dest', 'port_origin', 'port_name_origin',
       'code_origin', 'lat_origin', 'long_origin', 'nautical',
       'commercial'], dtype=object)

In [32]:
# Loc columns
cleandf = cleandf.loc[:, ['commercial','nautical','port_destination', 'port_origin', 'code_origin', 'code_destination']]
cleandf

,commercial,nautical,port_destination,port_origin,code_origin,code_destination
97,716,716,00739854-654f-469c-b5af-31680f7ae36e,13ae6ff9-3a38-42f4-bf51-ac03c0492d30,952,662
98,1035,1035,00739854-654f-469c-b5af-31680f7ae36e,0c87bc40-0218-4bc1-bbd8-3dfff430a3d9,845,662
99,1082,1082,00739854-654f-469c-b5af-31680f7ae36e,ddac0441-65ff-4960-b993-87c73ebc2d85,322,662
100,612,612,00739854-654f-469c-b5af-31680f7ae36e,9c1e061d-9b9f-404d-a4cc-c98db3337a64,951,662
103,659,659,00739854-654f-469c-b5af-31680f7ae36e,b2811ee3-48f4-41a3-b82a-bcd88ef3233c,946,662
...,...,...,...,...,...,...
9307,1786,1786,1334ec95-cc8a-4464-a263-0b0beee559da,c069f8c4-4a93-4021-9a75-c9aadb3b4a1c,947,251
9308,2549,2549,1334ec95-cc8a-4464-a263-0b0beee559da,3bd1c07f-44b1-4fb8-becc-e6faa9882fb3,979,251
9309,2011,2011,1334ec95-cc8a-4464-a263-0b0beee559da,4746d329-6f16-4e45-9cbc-7245ef40d5d6,725,251
9310,1575,1575,1334ec95-cc8a-4464-a263-0b0beee559da,72346087-6a6a-462b-a434-913d6f2e5ca9,942,251


In [ ]:
cleandf.to_csv('\data/port_distance.csv')
